# RNA Velocity analysis using scVelo

In [ ]:
import scanpy as sc
import scvelo as scv
import numpy as np
import glob
import os
import re

In [ ]:
# specify project directory
proj_dir = '/Users/jonrob/Documents/NBIS/LTS_projects/d_angeletti_1910/scMouseBcellFlu'

In [ ]:
# specify figure params
scv.set_figure_params(dpi=80, frameon=False, figsize=(7, 7), facecolor='white')

## Load and preprocess RNA velocity data (.loom files) 

In [ ]:
# specify directory containing RNA velocity .loom files
loom_dir = proj_dir + '/data/rna_velocity/loom_files/'

In [ ]:
# get list of directories in loom_dir (only those containing underscore "_")
loom_subdirs = glob.glob(loom_dir + '*_*')

# initialize loom_data with first file
sd = loom_subdirs.pop(0)
loom_data = scv.read_loom(glob.glob(sd + '/*.loom')[0])
loom_data.var_names_make_unique()
loom_data.obs['dataset'] = os.path.split(sd)[1]


In [ ]:
# append other datasets to loom_data
for sd in loom_subdirs:
    loom_files = glob.glob(sd + '/*.loom')
    if len(loom_files) == 1:
        ldat = scv.read_loom(loom_files[0])
        ldat.var_names_make_unique()
        ldat.obs['dataset'] = os.path.split(sd)[1]
        loom_data = loom_data.concatenate(ldat, join='outer', index_unique=None)
    elif len(loom_files) > 1:
        raise NameError('Each directory should only contain one loom file!')
    else:
        print('No loom files found in ' + os.path.split(sd)[1])

del loom_data.obs['batch']  # remove unneeded obs field

In [ ]:
# rename cells to be consistent with naming in RNA-Seq data (barcode seq + dataset)
loom_data.obs_names = [re.split(':|x', x)[1] + '_' + loom_data.obs['dataset'][x] for x in loom_data.obs_names]

## Load RNA-Seq data and merge with RNA velocity data

In [ ]:
# specify path and filename of AnnData object containing RNA-Seq data
ann_file = proj_dir + '/analysis/06_cluster/anndata_object_VDJannot.h5ad'
#ann_file = proj_dir + '/analysis/06_cluster/anndata_object_noplasma.h5ad'
#ann_file = proj_dir + '/analysis/trajectory_slingshot/trajectory_05/seurat_object_VDJannot_tr5.h5ad'

In [ ]:
# load scRNA-Seq AnnData object
adata = sc.read_h5ad(ann_file)

In [ ]:
# merge scRNA-Seq data with RNA velocity data (spliced and unspliced counts)
adata = scv.utils.merge(adata, loom_data)

In [ ]:
# OPTIONAL: Subset data to only include specified clusters
adata = adata[adata.obs['HC_16'].isin([2,3,4,5,8,9,10,11,12,13,14,15,16])]

## RNA velocity preprocessing and calculation

In [ ]:
# compute first and second order moments for velocity estimation
scv.pp.moments(adata, use_rep='mnn_cell_embeddings', n_neighbors=15)

In [ ]:
# Recover the full splicing kinetics of the genes (only necessary for "dynamical" velocity mode below)
#scv.tl.recover_dynamics(adata, plot_results=True)

In [ ]:
# compute cell velocities
scv.tl.velocity(adata, mode='stochastic')
#scv.tl.velocity(adata, mode='dynamical')

In [ ]:
# compute velocity graph based on cell cosine similarities
scv.tl.velocity_graph(adata)

## Visualization of cell velocities and pseudotime

In [ ]:
# visualize the velocity graph on the embedding
scv.pl.velocity_graph(adata, basis='umap_cell_embeddings', threshold=0.5, color='HC_16', edge_width=0.05)

In [ ]:
# generate stream plot of cell velocities on the embedding
scv.pl.velocity_embedding_stream(adata, basis='umap_cell_embeddings',
                                 color='HC_16', size=10, alpha=1, min_mass=0)

In [ ]:
# visualize velocity and expression of some specific genes
scv.pl.velocity(adata, basis='umap_cell_embeddings', var_names=['Ighd','Ighg1','Ighm','Igha','Aicda'])

In [ ]:
# visualize some additional features
scv.pl.scatter(adata, basis='umap_cell_embeddings', color=['MU_FREQ_TOT'], vmax=0.04, size=50, alpha=0.5)

In [ ]:
# visualize velocity magnitude and confidence
scv.tl.velocity_confidence(adata)
keys = ['velocity_length', 'velocity_confidence']
scv.pl.scatter(adata, c=keys, cmap='coolwarm', perc=[5, 95], basis='umap_cell_embeddings')

In [ ]:
# simulate the predicted descendents originating from a specified cell
root_cell = [i for i, x in enumerate(adata.obs['HC_16']) if x == '3'][0]  # choose cell in cluster 3 as root cell
x, y = scv.utils.get_cell_transitions(adata, basis='umap_cell_embeddings', starting_cell=root_cell, n_steps=1000, random_state=42)
ax = scv.pl.velocity_graph(adata, c='lightgrey', edge_width=.05, show=False, basis='umap_cell_embeddings')
ax = scv.pl.scatter(adata, x=x, y=y, s=50, c='ascending', cmap='gnuplot', ax=ax, basis='umap_cell_embeddings')

In [ ]:
# visualize velocity pseudotime on embedding
scv.tl.velocity_pseudotime(adata, root=root_cell)
scv.pl.scatter(adata, color='velocity_pseudotime', cmap='gnuplot', basis='umap_cell_embeddings')

In [ ]:
# visualize velocity latent time on embedding (must first run scv.tl.recover_dynamics above!)
adata.uns['iroot'] = root_cell
scv.tl.latent_time(adata, root_key='iroot')
scv.pl.scatter(adata, color='latent_time', cmap='gnuplot', basis='umap_cell_embeddings')

## PAGA velocity graph

In [ ]:
# Run PAGA
# this is needed due to a current bug
adata.uns['neighbors']['distances'] = adata.obsp['distances']
adata.uns['neighbors']['connectivities'] = adata.obsp['connectivities']

scv.tl.paga(adata, groups='HC_16')
df = scv.get_df(adata, 'paga/transitions_confidence', precision=2).T
df.style.background_gradient(cmap='Blues').format('{:.2g}')


In [ ]:
# visualize the PAGA velocity graph on the embedding
scv.pl.paga(adata, basis='umap_cell_embeddings', color='HC_16', size=10, alpha=0.5,
            min_edge_width=2, node_size_scale=2, legend_loc='on data', node_size_power=1)